# Alguns métodos de resolução de sistemas lineares

# Decomposição de Cholesky

###### Trata-se de uma simplificação da decomposição LU quando a matriz é simétrica positiva definida. A matriz $U$ é obtida através da transposição da matriz $L$.

### Temos então $L = 
\begin{bmatrix}
l_{11} & 0 & 0\\
l_{21} & l_{22} & 0\\
l_{31} & l_{32} & l_{33}\\
\end{bmatrix}$
### e
$U = \begin{bmatrix}
l_{11} & l_{21} & l_{31}\\
0 & l_{22} & l_{32}\\
0 & 0 & l_{33}\\
\end{bmatrix} = L^{t}$

### Para uma matriz $M$, $M = LU$, segue:

 
$M = \begin{bmatrix}
m_{11} & m_{12} & m_{13}\\
m_{21} & m_{22} & m_{23}\\
m_{31} & m_{32} & m_{33}\\
\end{bmatrix} = LU = LL^{t} = \begin{bmatrix}
l_{11}^{2} & l_{11}*l_{21} & l_{11}*l_{31}\\
l_{21}*l_{11} & l_{21}^{2} + l_{22}^{2} & l_{21}*l_{31}+l_{22}*l_{32}\\
l_{31}*l_{11} & l_{31}*l_{21} + l_{32}*l_{22} & l_{31}^{2} + l_{32}^{2} + l_{33}^{2}\\
\end{bmatrix}$

### Pseudocódigo desse troço (inicializando L com 0):
##### Para 
$k = 1:\\
l_{11} = \sqrt{m_{11}}\\
l_{i1} = \frac{m_{i1}}{l_{11}},  \forall i = 2...n$
##### Para
$ k = 2...n:\\
l_{kk} = [m_{kk} - \sum_{r=1}^{k-1} l_{kr}^{2}]^{1/2}\\
\text{Para } i = k+1...n:\\
l_{ik} = \frac{1}{l_{kk}}(m_{ik} - \sum_{r=1}^{k-1} l_{ir}*l_{rk})$
             

### Vamos começar com funções separadas para facilitar o entendimento e depois juntamos tudo numa maior.

In [1]:
import numpy as np
import pandas as pd

In [2]:
n = 3 ## tamanho da matriz de exemplo

In [3]:
M = np.zeros((n,n+1)) ##seta uma matriz de exemplo; a coluna neste método tem que ter dimensão n+1 para entrar a resposta
L = np.zeros((n, n+1)) ### matriz inferior com espacinho para a resposta

In [4]:
M

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [5]:
###preenchendo com os coeficientes e valores da matriz M
M[0][0] = 2
M[0][1] = -1
M[0][2] = 0
M[0][3] = 1
M[1][0] = -1
M[1][1] = 2
M[1][2] = -1
M[1][3] = 2
M[2][0] = 0
M[2][1] = -1
M[2][2] = 2
M[2][3] = 0

##A matriz L também passa por pivotação, então temos que deixar o valor da resposta nela

L[0][3] = 1
L[1][3] = 2
L[2][3] = 0


In [ ]:
M_orig = M.copy()

### Algoritmo de decomposição de Cholesky

In [ ]:
L[0,0] = np.sqrt(M[0,0])
L[:,0] = M[:,0]/L[0,0]

for k in range(1,n):
    ###diagonal principal
    sum_k = 0
    for r in range(k):
        sum_k+=L[k,r]**2
    L[k,k] = np.sqrt(M[k, k] - sum_k)
    
    for i in range(k+1, n):
        sum_i = 0
        for r in range(k-1):
            sum_i += L[i,r]*L[r,k]
        L[i,k] = (1/L[k,k])*(M[i,k] - sum_i)

In [ ]:
np.matmul(L[:,0:n], L[:,0:n].T)## Deu certo!

In [ ]:
[M, L, L.T]

### Armazenando a transposta

In [ ]:
Lt = np.zeros((n, n+1))

In [ ]:
Lt[:,0:n] = L.T[0:n:,0:n]

In [ ]:
Lt

### Precisamos agora resolver o problema LC =  B. Vamos olhar a matriz L:

In [ ]:
pd.DataFrame(L)

### O algoritmo de solução deste sistema é o de substituições sucessivas (exatamente o inverso da retrossubstituição sucessiva).


$\begin{equation}
\left[
    \begin{array}{ccc:c}
        L[0,0]x_1 & 0 & 0 & L[0,3]\\
        L[1,0]x_1 & L[1,1]x_2 & 0 & L[1,3]\\
        L[2,0]x_1 & L[2,1]x_2 & L[2,2]x_3 & L[2,3]\\
    \end{array}
\right]
\end{equation}$

### Neste caso, teríamos:

$\begin{equation}
x_1 = \frac{L[0,3]}{L[0,0]}\\
x_2  = \frac{L[1,3]}{L[1,1]} - \frac{L[1,0]*x_1}{L[1,1]}\\
x_3  = \frac{L[2,3]}{L[2,2]} - \frac{L[2,0]*x_1}{L[2,2]} - \frac{L[2,1]*x_2}{L[2,2]}
\end{equation}$

### Note que os coeficientes que acompanham as incógnitas $x_i$ andam até o limite (i) nas colunas; ex: na linha 1, 0 passos; na linha 2, 1 passo (até L[1,0]); na linha 3, 2 passos (até L[2,1]). 

In [ ]:
c = np.zeros((n,1)) ##vetor que vai receber a resposta

In [ ]:
### começamos com i = 0; andamos de cima pra baixo na matriz
c[0] = L[0,n]/L[0,0]

In [ ]:
for i in range(1, n): ###restringimos os casos a partir de 1, pois i = 0 ja foi calculado
    termo_ind = L[i,n]/L[i,i] ###termo independente da equacao
    soma = 0
    for j in range(i):           ###esta parte do loop é a mais crucial de ser entendida; escreva numa folhinha de papel o que ela faz que fica facil de enxergar
        soma += ((L[i,j])/(L[i,i]))*c[j]
    c[i] = termo_ind - soma          ####o vetor de respostas vai sendo alimentado do começo ao fim

In [ ]:
c

### Agora vamos resolver o problema $L^{t}X = C$

### Como $L^{t}$ é triangular superior, conseguimos aproveitar o algoritmo de retrossubstituições sucessivas:

In [ ]:
x = np.zeros((n, 1)) ##vetor que vai receber a solução do sistema 

In [ ]:
Lt[:,n] = c.reshape(1,-1) ###atribuindo o valor de C na última coluna de L^t

In [ ]:
## começamos com o caso de i = n (andamos de baixo pra cima na matriz)
x[n-1] = Lt[n-1, n]/Lt[n-1, n-1]

In [ ]:
for i in range(1, n): ###restringimos os casos a partir de n-1. o indexador do python ja fa isso naturalmente [1,n) ==[1,n-1]
    termo_ind = Lt[n-i-1,n]/Lt[n-i-1,n-i-1] 
    soma = 0
    for j in range(n-i, n):           ###esta parte do loop é a mais crucial de ser entendida; escreva numa folhinha de papel o que ela faz que fica facil de enxergar
        soma += ((Lt[n-i-1,j])/(Lt[n-i-1,n-i-1]))*x[j]
    x[n-i-1] = termo_ind - soma          ####o vetor de respostas vai sendo alimentado de tras pra frente tambem

In [ ]:
x

In [ ]:
pd.DataFrame(np.dot(M_orig[:,0:n],x).round(3)) #Sim! Deu certo. 

# Agrupando tudo em funções

In [6]:
def decomposicao_cholesky(M, n):
    L = np.zeros((M.shape))
    Lt = np.zeros((M.shape))
    
    L[0,0] = np.sqrt(M[0,0])
    L[:,0] = M[:,0]/L[0,0]

    for k in range(1,n):
        ###diagonal principal
        sum_k = 0
        for r in range(k):
            sum_k+=L[k,r]**2
        L[k,k] = np.sqrt(M[k, k] - sum_k)
    
        for i in range(k+1, n):
            sum_i = 0
            for r in range(k-1):
                sum_i += L[i,r]*L[r,k]
            L[i,k] = (1/L[k,k])*(M[i,k] - sum_i)
    Lt[0:n,0:n] = L[0:n,0:n].T
    L[:,n] = M[:,n]
    return [M, L, Lt]

In [7]:
sols = decomposicao_cholesky(M,n)
sols

[array([[ 2., -1.,  0.,  1.],
        [-1.,  2., -1.,  2.],
        [ 0., -1.,  2.,  0.]]),
 array([[ 1.41421356,  0.        ,  0.        ,  1.        ],
        [-0.70710678,  1.22474487,  0.        ,  2.        ],
        [ 0.        , -0.81649658,  1.15470054,  0.        ]]),
 array([[ 1.41421356, -0.70710678,  0.        ,  0.        ],
        [ 0.        ,  1.22474487, -0.81649658,  0.        ],
        [ 0.        ,  0.        ,  1.15470054,  0.        ]])]

In [8]:
def solucao(L, n):
    c = np.zeros((n, 1))
    c[0] = L[0,n]/L[0,0]
    for i in range(1, n): ###restringimos os casos a partir de 1, pois i = 0 ja foi calculado
        termo_ind = L[i,n]/L[i,i] ###termo independente da equacao
        soma = 0
        for j in range(i):           ###esta parte do loop é a mais crucial de ser entendida; escreva numa folhinha de papel o que ela faz que fica facil de enxergar
            soma += ((L[i,j])/(L[i,i]))*c[j]
        c[i] = termo_ind - soma        ####o vetor de respostas vai sendo alimentado do começo ao fim
    return c
    

In [9]:
c = solucao(sols[1], n)

In [10]:
def retro_solucao(U, n, c):
    x = np.zeros((n, 1))
    U[:,n] = c.reshape(1, -1)
    
    x[n-1] = U[n-1, n]/U[n-1, n-1] ## começamos com o caso de i = n (andamos de baixo pra cima na matriz)
    
    for i in range(1, n): ###restringimos os casos a partir de n-1. o indexador do python ja fa isso naturalmente [1,n) ==[1,n-1]
        termo_ind = U[n-i-1,n]/U[n-i-1,n-i-1] 
        soma = 0
        for j in range(n-i, n):           ###esta parte do loop é a mais crucial de ser entendida; escreva numa folhinha de papel o que ela faz que fica facil de enxergar
            soma += ((U[n-i-1,j])/(U[n-i-1,n-i-1]))*x[j]
        x[n-i-1] = termo_ind - soma          ####o vetor de respostas vai sendo alimentado de tras pra frente tambem
    return x

In [11]:
X = retro_solucao(sols[2], n, c)

In [12]:
c

array([[0.70710678],
       [2.04124145],
       [1.44337567]])

In [13]:
X

array([[1.75],
       [2.5 ],
       [1.25]])

### Testando se funcionou:

In [14]:
np.dot(M[:,0:n], X).round(3)  ##sim deu certo!

array([[1.],
       [2.],
       [0.]])